# PyTorch DeepLab の学習済みモデルを SageMaker でデプロイする

PyTorch Hub で公開されている [DeepLab V3 のモデル](https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/)をダウンロードしてデプロイします。このノートブックでは、モデルのSageMaker への持ち込み方法を知るため、以下のステップでデプロイします。

1. PyTorch Hub からモデルをダウンロードし、S3 に保存します。
1. ダウンロードしたモデルで推論を行うためのコードを作成します。
1. S3 に保存したモデルを指定して、SageMaker にデプロイします。

実際には、推論コードの中でPyTorch Hub からモデルをダウンロードできるため、1をスキップする方法も可能です。


## 1. PyTorch Hub からのモデルダウンロード

`torch.hub`でモデルをダウンロードし、パラメータの情報のみ保存します。保存したファイルは `tar.gz` の形式にして S3 にアップロードします。

In [ ]:
import torch
import os
os.makedirs('model',exist_ok=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True, progress=False)
path = './model/model.pth'
torch.save(model.cpu().state_dict(), path)

In [ ]:
!tar cvzf model.tar.gz -C ./model .

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
model_path = sagemaker_session.upload_data("model.tar.gz", key_prefix ="pytorch_deeplab_model")

In [ ]:
model_path

## 2. 推論コードの作成

アップロードしたモデルを読み込んで推論を実行するコードを作成します。モデルの読み込みは `model_fn` で、推論の実行は `transform_fn`で実装します。
PyTorch ではモデルのパラメータ以外にシンボルの情報が必要なので、PyTorch Hub から呼び出して利用します。各関数の実装は[公式の利用方法](https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/)を参考にしています。

In [ ]:
%%writefile deploy.py

from io import BytesIO
import json
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
from torchvision import transforms    

def model_fn(model_dir):
    model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=False, progress=False)
    with open(os.path.join(model_dir, "model.pth"), "rb") as f:
        model.load_state_dict(torch.load(f), strict=False)
    model.eval() # for inference
    return model

def transform_fn(model, request_body, request_content_type, response_content_type):
    
    input_data = np.load(BytesIO(request_body))
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    input_tensor = preprocess(input_data)
    input_batch = input_tensor.unsqueeze(0)
    prediction = model(input_batch)
    return json.dumps(prediction['out'].tolist())

## 3. デプロイと推論

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

deeplab_model=PyTorchModel(model_data=model_path, 
             role=sagemaker.get_execution_role(), 
             entry_point='deploy.py', 
             framework_version='1.8.1',
             py_version='py3')

In [ ]:
predictor=deeplab_model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

In [ ]:
!wget https://github.com/pytorch/hub/raw/master/images/dog.jpg

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torchvision import transforms
from skimage.segmentation import mark_boundaries

input_image = Image.open("dog.jpg").convert('RGB')
w, h = input_image.size

input_image  = input_image.resize((150, 100))
np_input_image = np.array(input_image)
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()
predictions = predictor.predict(np_input_image)

# create a color pallette, selecting a color for each class
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")
label_map = np.array(predictions[0]).argmax(0)

# # plot the semantic segmentation predictions of 21 classes in each color
r = Image.fromarray(label_map.astype(np.uint8))
r.putpalette(colors)
r = Image.blend(r.convert('RGBA'), input_image.convert('RGBA'), 0.5) 

plt.rcParams['figure.figsize'] = [12, 8]
plt.imshow(r)

最後に不要なエンドポイントを削除します。

In [ ]:
predictor.delete_endpoint()